## Word Embedding

En este notebook se creará un modelo de Word Embedding utilizando la librería Gensim. Se utilizará el dataset de entrenamiento de la competencia para entrenar el modelo. Luego, se utilizará el modelo para obtener los embeddings de las palabras de los textos de los tweets y se guardarán en un archivo para utilizarlos en los modelos de clasificación.

<div style="text-align: center">
    <img src="https://miro.medium.com/v2/resize:fit:678/1*5F4TXdFYwqi-BWTToQPIfg.jpeg" width="500">
</div>

In [ ]:
# Instalación de librerías
%pip install -r '../requirements.txt'

In [ ]:
# Importación de librerías
import pandas as pd
import json
from gensim.models import Word2Vec

In [ ]:
# Carga de datos de entrenamiento
# Se utilizará el dataset de entrenamiento para entrenar el modelo
dataset_entrenamiento = pd.read_csv('../3-NLP/dist/dataset-spanish-tweets-train-preprocessed.csv', sep=',', encoding='utf-8')
dataset_entrenamiento['text'] = dataset_entrenamiento['text'].apply(lambda x: json.loads(x))
dataset_entrenamiento.head()

In [ ]:
# Eliminar arreglos con longitud 0
dataset_entrenamiento = dataset_entrenamiento[dataset_entrenamiento['text'].apply(lambda x: len(x) > 0)]

In [ ]:
dataset_entrenamiento.info()

### Definición del Word Embedding

El Word Embedding es una técnica de procesamiento de lenguaje natural que consiste en asignar una representación vectorial a cada palabra de un vocabulario. Esta representación vectorial se obtiene a partir de un modelo de lenguaje entrenado con un corpus de texto. Los vectores obtenidos tienen la propiedad de que palabras similares en el contexto de los textos tienen vectores similares.

La idea es vectorizar cada una de las palabras en un vector de 150 dimensiones, luego hacer un promedio de los vectores de todas las palabras de un tweet para obtener un vector de 150 dimensiones que represente al tweet.

Cada dimensión del vector representa una característica de la palabra, por lo que palabras similares tendrán vectores similares.

Nuestro conjunto de datos al final será una matriz de 150 columnas, donde cada fila representa un tweet. Y por supuesto otra columna con la etiqueta del tweet.

### Hiperparámetros

1. sentences: lista de listas de palabras. Cada lista de palabras representa un tweet.
2. vector_size: tamaño del vector de embeddings. Aquí se define el número de dimensiones del vector de embeddings. Cuanto mayor sea el número, más información se podrá capturar, pero también será más costoso computacionalmente.
3. window: tamaño de la ventana de contexto. Es el número de palabras que se consideran a la izquierda y a la derecha de la palabra objetivo.
4. min_count: número mínimo de veces que una palabra debe aparecer en el corpus para ser considerada en el modelo.
5. workers: número de núcleos de la CPU a utilizar para entrenar el modelo.
6. sg: algoritmo de entrenamiento. 1 para Skip-gram, 0 para CBOW.
7. epochs: número de iteraciones sobre el corpus.
8. seed: semilla

In [ ]:
sentences = dataset_entrenamiento['text'].tolist()
vector_size = 150
window = 10
min_count = 1
workers = 4
sg = 1 # Skip Gram tiene mejor rendimiento frente a CBOW
epochs = 60
seed = 42 # Para reproducibilidad
model = Word2Vec(sentences=sentences,
                 vector_size=vector_size,
                 window=window,
                 min_count=min_count,
                 workers=workers,
                 sg=sg,
                 epochs=epochs,
                 seed=seed)

In [ ]:
# Una vez entrenado el modelo, se guarda en disco
model.save('./dist/word2vec.model')